<a href="https://colab.research.google.com/github/J-o-y-y/ms_ai_school/blob/main/20230612_%EB%8B%A4%EC%A4%91_%EA%B0%9D%EC%B2%B4_%EC%B6%94%EC%A0%81%EC%9D%84_%EC%9C%84%ED%95%9C_%EC%98%81%EC%83%81_%EC%B2%98%EB%A6%AC_%EA%B8%B0%EB%B2%95_%EB%B9%84%EA%B5%90_%EC%B2%AB%EB%B2%88%EC%A7%B8_%ED%8F%89%EA%B7%A0_%EC%9D%B4%EB%8F%99_%EC%B6%94%EC%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[20230612_다중 객체 추적을 위한 영상 처리 기법 비교]


평균 이동 추적
1. 초기 사각형 값 설정 후 동영상을 열고 추적할 객체에 대한 박스를 첫 프레임에서 생성합니다.
2. 박스로 지정한 부분을 잘라내고 HSV 색상 공간으로 변환한 후, 해당 부분의 히스토그램 값을 계산합니다. 계산된 히스토그램 값을 0~255 사이 값으로 정규화합니다.
3. While 문을 사용하여 프레임을 읽어오며, 각 프레임마다 HSV 색상으로 변환하고 히스토그램 역투영을 계산하여 잘라낸 박스와 유사한 부분을 찾습니다. 이후 시각화를 위해 박스를 생성합니다.



In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# 초기 사각형 값 설정
x, y, w, h = (100, 100, 50, 50)

# 동영상 파일 열기
cap = cv2.VideoCapture('slow_traffic_small.mp4')

# 첫 프레임에서 추적할 객체 선택
ret, frame = cap.read()
selected_roi = frame[y:y+h, x:x+w]

# HSV로 변환하여 히스토그램 계산
hsv_roi = cv2.cvtColor(selected_roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# 추적할 객체의 초기 윈도우 설정
track_window = (x, y, w, h)

# 평균 이동 알고리즘을 위한 설정
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break
    
    # HSV로 변환
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # 히스토그램 역투영 계산
    back_projection = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
    
    # 평균 이동 알고리즘을 통해 객체 위치 추정
    ret, track_window = cv2.meanShift(back_projection, track_window, term_crit)
    
    # 추적 결과를 사각형으로 표시
    x, y, w, h = track_window
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # 결과 출력
    cv2_imshow(frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()


동영상 파일인 "slow_traffic_small.mp4"을 열고 첫 프레임에서 추적할 객체에 대한 박스를 선택하여 초기 사각형을 설정합니다. 그 후, 선택한 객체의 히스토그램을 계산하고 추적할 객체의 초기 윈도우를 설정합니다.

그 다음, 동영상의 각 프레임에 대해 객체의 히스토그램 역투영을 계산하고 평균 이동 알고리즘을 통해 객체의 위치를 추정합니다. 추정된 위치를 바탕으로 사각형을 그려주고, 결과 영상을 출력합니다. 마지막으로 'q' 키를 누르면 종료하고, 동영상 파일을 닫고 창을 닫습니다.

출력되는 좌표는 추적된 객체의 위치를 나타내며, x, y, w, h 순서로 좌표 값이 출력됩니다.